# Spam Classifier using Bayes Theorem

In [2]:
%%time
import pandas as pd
import numpy as np 
import string
import re
from collections import Counter
import io
import requests

CPU times: user 1.22 s, sys: 168 ms, total: 1.39 s
Wall time: 6.23 s


### Dataset from Kaggle

In [ ]:
mainData = pd.DataFrame(pd.read_csv('spamraw.csv'))

This Data contains two columns
Text and Type
Text contains email/message and type contains its answer whether it is spam or ham

In [6]:
print(mainData.iloc[0])

type                                                  ham
text    Hope you are having a good week. Just checking in
Name: 0, dtype: object


In [8]:
trainingData,testData = np.split(mainData,[int(0.7*len(mainData))])

# Working on Training Data from here

#### Seperating out Spam and Ham Data

In [9]:
spamTraining = trainingData[trainingData['type']=='spam']
hamTraining = trainingData[trainingData['type']=='ham']

#### Then appending whole Spam text and Ham text into a single String
Doing this will make it easier to calculate relative frequencies of all the words 

In [10]:
wholeDataSpam = ''
wholeDataHam = ''

for i in range(0,len(spamTraining)):
    temp = spamTraining['text'].iloc[i]
    temp.lower()
    temp = re.sub(r'[^\w\s]','',temp)
    wholeDataSpam = wholeDataSpam + " " + temp
    

for i in range(0,len(hamTraining)):
    temp = hamTraining['text'].iloc[i]
    temp.lower()
    temp = re.sub(r'[^\w\s]','',temp)
    wholeDataHam = wholeDataHam + " " + temp



#### Getting Relative Frequencies in Dictionary

In [11]:
spamFrequency = dict(Counter(wholeDataSpam.split()))
hamFrequency = dict(Counter(wholeDataHam.split()))

In [12]:
spamValues = list(spamFrequency.values())
hamValues = list(hamFrequency.values())
hamKeys = list(hamFrequency.keys())
spamKeys = list(spamFrequency.keys())
spamValues = np.array(spamValues)
spamValues  = spamValues/len(spamValues)
hamValues = np.array(hamValues)
hamValues = hamValues/len(hamValues)

spamRF = dict(zip(spamKeys,spamValues))
hamRF = dict(zip(hamKeys,hamValues))


# Testing Area

In this section I am calculating prior log likeilihood probabilities of Spam and Ham then using those in Bayes Rule Relation along with p(Spam) and p(Ham)
This will give me probability whether an Email Message is Spam as well as Ham
And then I am compairing both Probabilities and greater one is the answer.

In [13]:
priorProbabilitySpam = 0 
priorProbabilityHam = 0
answers = []
correct = 0

pSpam = np.log(len(spamTraining)/len(trainingData))
pHam = np.log(len(hamTraining)/len(trainingData))

for i in range(0,len(testData)):
    temp = testData['text'].iloc[i]
    temp = temp.lower()
    temp = re.sub(r'[^\w\s]','',temp)
    temp = temp.split()
    
    commonSpam = list(set(temp) & set(spamRF.keys()))
    commonHam = list(set(temp) & set(hamRF.keys()))
    
    for k in range(0,len(commonSpam)):
        priorProbabilitySpam = priorProbabilitySpam + np.log(spamRF[commonSpam[k]])
    for a in range(0,len(commonHam)):
        priorProbabilityHam = priorProbabilityHam + np.log(hamRF[commonHam[a]])
        
    finalBayesSpam = (priorProbabilitySpam*pSpam)/(pHam)
    finalBayesHam = (priorProbabilityHam*pHam)/(pSpam)

    
    if(finalBayesSpam>finalBayesHam):
        answers.append('spam')
    else:
        answers.append('ham')
        
        
for i in range(0,len(answers)):
    if(answers[i] == testData['type'].iloc[i]):
        correct = correct+1
        
        
print("=================================================")
print("Correct Answers: ",correct,"Out of:",len(answers))
print("=================================================")

accuracy = (correct/len(answers))*100
print("=================================")
print("TestingAccuracy: ",accuracy,"%")
print("=================================")


Correct Answers:  1447 Out of: 1668
TestingAccuracy:  86.7505995203837 %
